# Aula 4 - Manipulação de df: groupby e merge


### Objetivos

Apresentar como unir dataframes e realizar cálculos com dados agrupados

____________________________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como concatenar dataframes,
- Conseguir agrupar os dados e aplicar vários métodos à eles


____
____
____

## Titanic

O arquivo que usaremos hoje é relativo ao Titanic! Essa é uma das bases mais famosas de ciência de dados. Você pode saber mais sobre estes dados [clicando aqui!](https://www.kaggle.com/c/titanic)

In [2]:
import pandas as pd
import numpy as np

In [3]:
# lê dataframe do arquivo titanic.csv 
df = pd.read_csv("data/titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


E se quisessemos calcular a média de Fare por Pclasse utilizando apenas o que aprendemos até agora?

Ou de forma mais automática:

In [ ]:
for i in df.Pclass.sort_values().unique():
    print(f'Ticket médio da classe {i}: ', df[df['Pclass']==i].Fare.mean())

E se quisessemos calcular a média por Pclass e Sex?

### Groupby
Assim como no SQL, no pandas também temos um método com o qual podemos agregar os dados. O `groupby` primeiro separa nossos dados em grupos definidos dentro do método,  aplica um tipo de operação usando agregação, transformação, filtragem ou até uma função própria e, por fim, junta os resultados encontrados.
<br>

<img src="groupby.png"  style="width: 700px" >

Exemplo de aplicação da função de agregação `mean`
<br><br><br>

Utilizar o `groupby` é o mesmo que fazer a sequência:

   1. Dividir os dados em grupos utilizando um critério
    
   2. Aplicar uma função em cada um dos grupos separadamente
    
   3. Combinar o resultado em uma estrutura de dados

#### Funções de agregação
Com essas funções podemos aplicar operações estatísticas nos nossos dados. Exemplos:<br>
`mean`, `std`, `max`, `min`, `count`, `sum`, `var`. <br>
Quando queremos aplicar apenas uma dessas operações podemos chamá-las diretamente após o `groupby`:


In [ ]:
# Agrupa por Pclass e Sex e calcula a média de cada grupo


Aqui agregamos os dados por Pclass e Sex e em todas as colunas numéricas foi calculada a média. Se quiséssemos a média de apenas uma coluna poderíamos adicioná-la ao final da nossa sentença:

In [ ]:
# Queremos apenas a média de idade considerando a classe e o sexo
df.groupby(["Pclass", "Sex"]).mean()[['Age']]

Ou de modo mais eficiente:

In [ ]:
df.groupby(["Pclass", "Sex"])[['Age']].mean()

Note que `df.groupby('A').colname.mean()` é mais eficiente que `df.groupby('A').mean().colname` pois a agregação só será realizada na coluna de interesse (colname).

Quando queremos aplicar mais de uma operação chamamos o método `.agg()`

In [ ]:
df.groupby(["Pclass"]).agg(['mean','max','min'])

Para operações distintas em colunas distintas passamos um dicionário com o nome da coluna como chave e a operação como valor

In [6]:
import numpy as np
df.groupby(['Pclass']).agg({'Embarked': pd.Series.mode, 'Fare': np.mean})

,Embarked,Fare
Pclass,,
1,S,84.154687
2,S,20.662183
3,S,13.675550


Reparem que a coluna utilizada no `groupby` virou um index do nosso df. Para convertê-la em coluna novamente temos duas formas: <br>
  1. chamar o parâmetro `as_index=False` dentro do `groupby`
  2. aplicar `.reset_index()` ao final da sentença

In [ ]:
# exemplo com as_index = False


In [ ]:
# exemplo com .reset_index()


_____________
_____________
**Exercício:** Existe diferença de sobrevivência por portão de embarque? E diferença no preço do ticket? Porque você acha que tem essa diferença?

In [136]:
# Resposta
df.groupby(['Embarked','Pclass'])[['Survived','Fare']].agg({'mean'})



Survived        Fare
                     mean        mean
Embarked Pclass                      
C        1       0.694118  104.718529
         2       0.529412   25.358335
         3       0.378788   11.214083
Q        1       0.500000   90.000000
         2       0.666667   12.350000
         3       0.375000   11.183393
S        1       0.582677   70.364862
         2       0.463415   20.327439
         3       0.189802   14.644083

______________
_____________

E se quiséssemos criar uma coluna nova que contenham o valor médio do Fare por Pclass?

### Criando coluna com dado agregado

Queremos que todas as pessoas da primeira classe tenham o valor 84.15 nessa nova coluna, todas da segunda classe tenham o valor 20.66 e da terceira classe 13.67. <br>
Podemos tentar:

In [ ]:
df.groupby('Pclass')[["Fare"]].mean()

In [ ]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].mean()

df.head(7)

Xiiii... deu ruim...
<br>
<br>


#### Transformação dos dados
Ao aplicarmos o método `.transform()` temos como retorno um objeto com o mesmo index do df de origem contendo a a transformação realizada para cada uma das linhas. Dessa forma podemos utilizar esse método e apenas criar uma coluna nova no nosso df.
<br>

Ele será muito **útil na criação de novas features** para os modelos.

In [ ]:
df.groupby('Pclass')[["Fare"]].transform('mean')

In [ ]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].transform('mean')
df.head(10)

Podemos aplicar tanto as operações mencionadas na agregação quanto uma função `lambda`:

In [ ]:
df['variacao_max_min'] = df.groupby('Pclass')[["Fare"]].transform(lambda x: x.max() - x.min())
df.head(10)

Ou até mesmo passar funções construídas:

In [ ]:
def funcao_max_menos_min(x):
    return x.max() - x.min()

In [ ]:
df.groupby('Pclass')[["Fare"]].transform(funcao_max_menos_min)

Também podemos preencher os valores nulos com a média de cada grupo

In [ ]:
# verificando quantidade de nulos por coluna
df.isna().sum()

Para preencher os nulos utilizaremos o método `.fillna()` que vimos em aula:

In [ ]:
df['Age_sem_nulo'] = df.groupby(['Pclass'])[['Age']].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df.isna().sum()

In [200]:
# Conferindo o preenchimento de nulos
# idade média por Pclass
df.groupby(['Pclass'])[['Age']].mean()

,Age
Pclass,
1,38.233441
2,29.877630
3,25.140620


In [ ]:
# selecionando a parte do df que tem idade nula
df[df.Age.isna()].head(10)

_________________________
_________________________
**Exercício:** Crie uma coluna com a média de Fare e outra com a média de idade para cada classe da coluna Survived. Você consegue fazer isso de uma única vez?

In [54]:
df.groupby(['Survived'])[['Fare','Age']].transform('mean')

,Fare,Age
0,22.117887,30.626179
1,48.395408,28.343690
2,48.395408,28.343690
3,48.395408,28.343690
4,22.117887,30.626179
...,...,...
886,22.117887,30.626179
887,48.395408,28.343690
888,22.117887,30.626179
889,48.395408,28.343690


In [55]:
# Resposta
df[['Mean_Fare','Mean_Age']] = df.groupby('Survived')[['Fare','Age']].transform('mean')

df.head(15)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Mean_Fare,Mean_Age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,22.117887,30.626179
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,48.395408,28.343690
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,48.395408,28.343690
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,48.395408,28.343690
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,22.117887,30.626179
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,22.117887,30.626179
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,22.117887,30.626179
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,22.117887,30.626179
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,48.395408,28.343690
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,48.395408,28.343690


_________________________
_________________________

## Cruzamento e concatenação de bases

Também é possível fazer **cruzamento de bases** com o pandas. 

Pra quem conhece SQL: esses são os joins!

Pra quem conhece Excel: essa é uma forma de fazer o procv!

Vamos supor que temos as notas de duas provas dos alunos separas em sheets diferentes do excel e queremos juntar essa notas em um único df.

In [118]:
# ler os dados de diferentes sheets do mesmo excel "notas.xlsx"
df1 = pd.read_excel("notas.xlsx", sheet_name="notas1")
df2 = pd.read_excel("notas.xlsx", sheet_name="notas2")

In [59]:
df1

,RA,aluno,prova1
0,1,joão,10
1,4,leia,10
2,2,maria,9
3,3,han,8
4,5,luke,7
5,7,obi wan,10


In [60]:
df2

,RA,aluno,prova2
0,1,joão,10
1,4,leia,10
2,2,maria,9
3,3,han,8
4,5,luke,7
5,6,anakin,10


Repare que temos alunos distintos nos dois df

Diferentes tipos de join

<img src="join_exemplo2.png" />
Fonte: https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78

O pandas possui dois métodos específicos para trabalharmos com o join de colunas entre df: `.merge()` e `.join()`. O `.merge()` fornece mais flexibilidade de trabalho e iremos utilizar e ele.

### pd.merge()
pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y")
)

In [119]:
df1.merge(df2, how="outer", on="RA")

,RA,aluno_x,prova1,aluno_y,prova2
0,1,joão,10.0,joão,10.0
1,4,leia,10.0,leia,10.0
2,2,maria,9.0,maria,9.0
3,3,han,8.0,han,8.0
4,5,luke,7.0,luke,7.0
5,7,obi wan,10.0,NaN,NaN
6,6,NaN,NaN,anakin,10.0


In [164]:
df1.merge(df2, how="outer", on=["RA", "aluno"])

,RA,aluno,prova1,prova2
0,1,joão,10.0,10.0
1,4,leia,10.0,10.0
2,2,maria,9.0,9.0
3,3,han,8.0,8.0
4,5,luke,7.0,7.0
5,7,obi wan,10.0,NaN
6,6,anakin,NaN,10.0


### pd.concat()
Diferente do `.merge()` e `.join()` que operam apenas com colunas, com o `.concat()` podemos especificar se queremos **concatenar em linhas ou colunas**.
Na concatenação de colunas o `.concat()` somente considera o index dos df e, por isso, não podemos especificar colunas como feito com o `.merge()`.

`pd.concat(
    objs,
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)`


In [165]:
pd.concat([df1, df2], axis=1, join="inner")

,RA,aluno,prova1,RA,aluno,prova2
0,1,joão,10,1,joão,10
1,4,leia,10,4,leia,10
2,2,maria,9,2,maria,9
3,3,han,8,3,han,8
4,5,luke,7,5,luke,7
5,7,obi wan,10,6,anakin,10


Repare que ao concatenar diretamente pelo index ele juntou o aluno obi wan com o anakin. 

Ao concatenar dois df nas linhas, o `.concat()` irá considerar o nome das colunas. Se temos colunas com nomes distintos e utilizamos o parâmetro join='inner', ele irá ignorar essas colunas: 

In [ ]:
pd.concat([df1, df2], axis=0, join="inner")

Para que ele considere todas as colunas utilizamos o argumento 
```python 
join="outer" 
```

In [ ]:
pd.concat([df1, df2], join="outer")

## Exercícios

1. Considere a existência de três tabelas distintas:
* customer.csv : Possui a informação dos clientes em duas colunas: customer id  customer name
* products.csv : Conté informação dos produtos vendidos pela empresa em três colunas - p_id (product id), product (name) e price
* sales.csv : Contém informações das vendas realizadas em seis colunas - sale_id, c_id (customer id), p_id (product_id), qty (quantity sold), store (name)

Conhecendo as bases e utilizando os métodos de concatenação de bases responda:


In [3]:
clientes = pd.read_csv('./data/customer.csv')
produtos = pd.read_csv('./data/products.csv')
vendas = pd.read_csv('./data/sales.csv')
produtos


,p_id,product,price
0,1,Hard Disk,80
1,2,RAM,90
2,3,Monitor,75
3,4,CPU,55
4,5,Keyboard,20
5,6,Mouse,10
6,7,Motherboard,50
7,8,Power supply,20


a) Quais produtos não foram vendidos?

In [6]:
nao_vendidos = produtos.merge(vendas, how='outer', on='product')


Series([], Name: product, dtype: object)

b) Quantos clientes não realizaram uma compra? 

In [7]:
clientes_nao_compraram = clientes.merge(vendas, how='outer', on='c_id')

clientes_nao_compraram[clientes_nao_compraram['sale_id'].isna()]['Customer']

9     King
10    Ronn
11     Jem
12     Tom
Name: Customer, dtype: object

c) Liste a quantidade vendida e o faturamento de cada produto 

In [8]:
venda_fatura = produtos.merge(vendas, how='inner', on='product')



venda_fatura['total_earnings'] = venda_fatura['qty'] * venda_fatura['price']



venda_fatura.groupby('product')[['qty','total_earnings']].sum().reset_index()






,product,qty,total_earnings
0,CPU,1,55
1,Monitor,12,900
2,RAM,7,630


d) Liste a quantidade vendida de cada produto por loja

In [13]:
venda_fatura.groupby(['store','product'])[['qty']].sum().reset_index()



,store,product,qty
0,ABC,Monitor,10
1,ABC,RAM,3
2,DEF,CPU,1
3,DEF,Monitor,2
4,DEF,RAM,4


e) Qual loja teve maior faturamento?

In [14]:
venda_fatura.groupby(['store'])[['total_earnings']].sum().reset_index()

,store,total_earnings
0,ABC,1020
1,DEF,565


f) Qual produto foi o mais vendido?

In [9]:
venda_fatura.groupby(['product'])[['qty']].sum().reset_index()

,product,qty
0,CPU,1
1,Monitor,12
2,RAM,7


## Referências
https://pandas.pydata.org/docs/user_guide/groupby.html <br>
https://pandas.pydata.org/docs/user_guide/merging.html <br> 
https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78 <br>
[When to use pandas transform function](https://towardsdatascience.com/when-to-use-pandas-transform-function-df8861aa0dcf)

## Material extra

### Outros parâmetros do groupby por default
* as_index
* sort
* dropna # exclui nans nas keys

<br> Em todas o default do python é True <br>
df.groupby('Pclass', sort=False)["Fare"].mean()

In [ ]:
# dropna
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

In [ ]:
# Default ``dropna`` is set to True, which will exclude NaNs in keys
df_dropna.groupby(by=["b"], dropna=True).sum()

In [ ]:
df_dropna.groupby(by=["b"], dropna=False).sum()

Repare que podemos chamar qualquer função do `pd.Series` ou  do `numpy`

In [ ]:
df.groupby(["Survived"]).mean()

### Função Lambda
Uma função lambda nada mais é que uma **forma alternativa de declarar uma função**, de um jeito mais direto

In [ ]:
# função que retorna o dobro de um número usando def
def dobro(x):
    
    return 2*x

dobro(2)

In [ ]:
# função que retorna o dobro de um número usando lambda x
faz_dobro = lambda x: 2*x

In [ ]:
faz_dobro(6)

### Apply
O método `.apply()` recebe uma função como input e aplica ela para todo o df como se fosse um loop. Se você quiser que essa função seja aplicada ao longo das colunas deve considerar axis=0 e ao longo das linhas axis=1)

In [ ]:
df.groupby(['Pclass']).apply(lambda x: x.describe())

Uma grande funcionalidade do pandas é que com o método `apply()` podemos aplicar uma **função** (muitas vezes, uma **função lambda**) a uma coluna ou linha de um DataFrame



Vamos selecionar a coluna de idades...

In [ ]:
df["Age"]

Aplicando uma função lambda **a todos os elementos da coluna**, ou seja, **à todas as linhas da tabela, daquela coluna específica**:

Tomando cada idade + 2, usando a função lambda definida.

Essa função lambda é equivalente a:

```python

def funcao(x):

    return x + 2
```

In [ ]:
df["Age"].apply(lambda x: x + 2)

In [ ]:
def funcao(x):
    return x + 2

df.Age.apply(funcao)

In [ ]:
df.Age.transform(funcao)

Um outro exemplo:

In [ ]:
# função: transforma todos os números em string, e concatena "!!!!!!!!!" à string
df["Age"].apply(lambda x: str(x) + "!!!!!!!!!")

Vamos usar uma função lambda para **extrair o sobrenome** dos nomes dos passageiros

Pra extrarir o sobrenome, note que este está separada do resto do nome por vírgula.

Para perceber isso, dê uma olhada na coluna de nomes:

In [ ]:
df["Name"]

Portanto, podemos usar a função para strings `split(",")`, com quebra na vírgula, e depois selecionar o primeiro elemento da lista gerada!

Vamos aproveitar e **criar uma nova coluna da base**, com os sobrenomes!

In [ ]:
df["Surname"] = df["Name"].apply(lambda x: x.split(",")[0])

In [ ]:
df["Surname"]

### Apply com funções

E se quisessemos comparar o quanto cada passageiro pagou a mais ou a menos da média do Fare?

In [ ]:
def f(group):
    return pd.DataFrame({'Fare_original': group,
                         'Fare_variacao': group - group.mean()})

df[['Fare_original','Fare_variacao']] = df.groupby('Pclass')['Fare'].apply(f)

In [ ]:
df.head()

#### Transform X Apply
Com uma função de agregação o `.transform()` retorna um df que tem a mesma quantidade de linhas que o df original enquanto o `.apply` retorna o agregado por grupos.

### Filtros
O filtro retorna apenas um subset do nosso df. Aqui podemos aplicar filtros mais elaborados do que os vistos na última aula. <br>
Podemos, por exemplo, eliminar categorias do df que possuem apenas alguns elementos:

In [ ]:
df.SibSp.value_counts()

In [ ]:
df.shape

In [ ]:
def filter_func(x):
    return x['Fare'] - x.Fare_Mean < 100

# df_filter = df.groupby(['SibSp']).filter(lambda x: filter_func(x))

df_filter = df.groupby(['SibSp']).filter(lambda x: len(x) >20)
df_filter.shape

In [ ]:
df_filter.SibSp.value_counts()

Vamos supor que antes de afundar o titanic, o time de hapiness quisesse promover uma jogatina para os grupos (segmentado por classe e sexo) que possuem idade média acima de 30 anos.

In [ ]:
df.groupby(['Pclass','Sex'])[['Age']].mean()

como podemos filtrar nosso df para termos apenas os passageiros que pertecem a essas segmentações escolhidas?

In [ ]:
df.groupby(['Pclass','Sex']).filter(lambda x: x['Age'].mean()>30)